In [1]:
# AI绘画亚洲人模型：chilloutmix

#第1步：加载程序包
%cd /kaggle/working/
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
!echo “加载完成”

/kaggle/working
Cloning into 'stable-diffusion-webui'...
remote: Enumerating objects: 17850, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 17850 (delta 0), reused 0 (delta 0), pack-reused 17846
Receiving objects: 100% (17850/17850), 28.23 MiB | 22.67 MiB/s, done.
Resolving deltas: 100% (12499/12499), done.
“加载完成”


In [2]:
#第2步：安装相关依赖库
# install Python 3.10(安装Python 3.10)
!conda create -n sdwebui -c cctbx202208 -y
!source /opt/conda/bin/activate sdwebui && conda install -q -c cctbx202208 python -y

# show gpu info
!nvidia-smi

# install xformers(安装xformers)
!/opt/conda/envs/sdwebui/bin/python3 -m pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 -f https://download.pytorch.org/whl/torch_stable.html
from subprocess import getoutput
s = getoutput('nvidia-smi')
if 'T4' in s:
    !/opt/conda/envs/sdwebui/bin/python3 -m pip install https://github.com/Testertime/xformers_T4_precompiled/releases/download/compiled/xformers-0.0.16+bc08bbc.d20230128-cp310-cp310-linux_x86_64.whl
elif 'P100' in s:
    !/opt/conda/envs/sdwebui/bin/python3 -m pip install https://github.com/Testertime/xformers_P100_precompiled/releases/download/compiled/xformers-0.0.16+bc08bbc.d20230128-cp310-cp310-linux_x86_64.whl

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda/envs/sdwebui



Preparing transaction: done
Verifying transaction: done
Executing transaction: done
#
# To activate this environment, use
#
#     $ conda activate sdwebui
#
# To deactivate an active environment, use
#
#     $ conda deactivate

Retrieving notices: ...working... done
Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda/envs/sdwebui

  added / updated specs:
    - python


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  cctbx202208
    _openmp_mutex-4.5          |            2_gnu          23 KB  cctbx202208

In [3]:
#第3步：调试环境与GPU
# temporary fix for now
!/opt/conda/envs/sdwebui/bin/python3 -m pip install -q --pre triton

# hacks to load model in gpu
!sed -i 's/map_location=None/map_location="cuda"/g' /kaggle/working/stable-diffusion-webui/modules/sd_models.py

# 使用最新版本的gradio时会有bug，无法正常工作，因此回退3.9版本
# 参考https://github.com/AUTOMATIC1111/stable-diffusion-webui/issues/6210
!sed -i 's/gradio==3.16.2/gradio==3.9.0/g' /kaggle/working/stable-diffusion-webui/requirements.txt

# 使用最新版本的fastapi时会有bug，无法正常工作
# 参考https://github.com/AUTOMATIC1111/stable-diffusion-webui/issues/7714
# 参考https://github.com/AUTOMATIC1111/stable-diffusion-webui/pull/7717
!/opt/conda/envs/sdwebui/bin/python3 -m pip install fastapi==0.90.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 59.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 9.8 MB/s eta 0:00:00


In [4]:
#第4步：拉取相关插件

%cd /kaggle/working/stable-diffusion-webui/extensions/

# tag自动补全插件（可选）
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git tag-autocomplete

# 下载审美渐变插件（可选）
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-aesthetic-gradients aesthetic-gradients
    
# 下载历史记录/图像浏览器插件（可选）
!git clone https://github.com/yfszzx/stable-diffusion-webui-images-browser images-browser
    
# 中文UI界面翻译插件（可选）
!git clone https://github.com/dtlnor/stable-diffusion-webui-localization-zh_CN localization-zh_CN
    
# lora networks插件（可选）
!git clone https://github.com/kohya-ss/sd-webui-additional-networks sd-webui-additional-networks

#controlnet插件（可选）
!git clone https://jihulab.com/hunter0725/sd-webui-controlnet sd-webui-controlnet

/kaggle/working/stable-diffusion-webui/extensions
Cloning into 'tag-autocomplete'...
remote: Enumerating objects: 1043, done.
remote: Counting objects: 100% (334/334), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 1043 (delta 219), reused 276 (delta 187), pack-reused 709
Receiving objects: 100% (1043/1043), 4.76 MiB | 21.87 MiB/s, done.
Resolving deltas: 100% (563/563), done.
Cloning into 'aesthetic-gradients'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 25 (delta 5), reused 21 (delta 4), pack-reused 0
Receiving objects: 100% (25/25), 1.10 MiB | 24.38 MiB/s, done.
Resolving deltas: 100% (5/5), done.
Cloning into 'images-browser'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 143 (delta 25), reused 22 (delta 22), pack-reused 109
Receiving objects: 100% (1

In [5]:
#第5步：连接模型并下载
# link models(连接模型)
%cd /kaggle/working/stable-diffusion-webui/models/Stable-diffusion/
!curl -Lo chilloutmixni.safetensors https://huggingface.co/nolanaatama/chomni/resolve/main/chomni.safetensors
!curl -Lo pastel-waifu-diffusion.vae.pt https://huggingface.co/andite/pastel-mix/resolve/main/pastel-waifu-diffusion.vae.pt
!echo “加载完成”

/kaggle/working/stable-diffusion-webui/models/Stable-diffusion
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1131  100  1131    0     0   2738      0 --:--:-- --:--:-- --:--:--  2738
100 7346M  100 7346M    0     0   280M      0  0:00:26  0:00:26 --:--:--  278M0     0   276M      0  0:00:26  0:00:18  0:00:08  195M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1153  100  1153    0     0   2839      0 --:--:-- --:--:-- --:--:--  2846
100  385M  100  385M    0     0   140M      0  0:00:02  0:00:02 --:--:--  191M
“加载完成”


In [6]:
#第6步：下载control_sd15_openpose.pth
%cd /kaggle/working/stable-diffusion-webui/extensions/sd-webui-controlnet/models
!curl -Lo control_sd15_openpose_2.pth https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_openpose.pth
!echo “加载完成”

/kaggle/working/stable-diffusion-webui/extensions/sd-webui-controlnet/models
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1145  100  1145    0     0   2843      0 --:--:-- --:--:-- --:--:--  2841
100 5446M  100 5446M    0     0   284M      0  0:00:19  0:00:19 --:--:--  293M0:20  0:00:07  0:00:13  285M
“加载完成”


In [7]:
#第7步：连接lora模型与相应插件

# link lora(连接lora)
%cd /kaggle/working/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora
!wget -O koreanDollLikenesss_v10.safetensors https://huggingface.co/AnonPerson/ChilloutMix/resolve/main/Korean-doll-likeness.safetensors
!wget -O chilloutmixss.safetensors https://civitai.com/api/download/models/19219
!wget -O tifaMeenow_tifaV2.safetensors https://civitai.com/api/download/models/16793
!wget -O Japanese-doll-likeness.safetensors https://huggingface.co/AnonPerson/ChilloutMix/resolve/main/Japanese-doll-likeness.safetensors
!wget -O Taiwan-doll-likeness.safetensors https://huggingface.co/AnonPerson/ChilloutMix/resolve/main/Taiwan-doll-likeness.safetensors
!wget -O 20denBlackPantyhosev10.safetensors https://civitai.com/api/download/models/17988
!wget -O cuteGirlMix4_v10.safetensors https://civitai.com/api/download/models/16677
!wget -O muvLuvsStyleArmors_10.safetensors https://civitai.com/api/download/models/13238
!wget -O chiArmorZhajiaV10.safetensors https://civitai.com/api/download/models/19299
!wget -O seeThroughSilhouette_v10.safetensors https://civitai.com/api/download/models/13189
!wget -O russianDollLikeness_v3.safetensors https://civitai.com/api/download/models/15396
!echo “加载完成”    


/kaggle/working/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora
--2023-03-30 15:19:48--  https://huggingface.co/AnonPerson/ChilloutMix/resolve/main/Korean-doll-likeness.safetensors
Resolving huggingface.co (huggingface.co)... 54.82.45.103, 52.2.178.255, 35.173.225.216, ...
Connecting to huggingface.co (huggingface.co)|54.82.45.103|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/60/ab/60abe86cb2039ff088635a1feff5c824cc79a13c96408fe3e88d25bb251d1b04/62efe75048d55a096a238c6e8c4e12d61b36bf59e388a90589335f750923954c?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27Korean-doll-likeness.safetensors%3B+filename%3D%22Korean-doll-likeness.safetensors%22%3B&Expires=1680448789&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzYwL2FiLzYwYWJlODZjYjIwMzlmZjA4ODYzNWExZmVmZjVjODI0Y2M3OWExM2M5NjQwOGZlM2U4OGQyNWJiMjUxZDFiMDQvNjJlZmU3NTA0OGQ1NWEwOTZhMjM

In [8]:
#第8步：开始运行
#启动程序
%cd /kaggle/working/stable-diffusion-webui
!COMMANDLINE_ARGS="--share --xformers --deepdanbooru --lowram --theme dark --disable-safe-unpickle --disable-nan-check --ckpt /kaggle/working/stable-diffusion-webui/models/Stable-diffusion/chilloutmix/chilloutmix_NiPrunedFp32Fix.safetensors" REQS_FILE="requirements.txt" /opt/conda/envs/sdwebui/bin/python3 launch.py

/kaggle/working/stable-diffusion-webui
Python 3.10.5 | packaged by conda-forge | (main, Jun 14 2022, 07:04:59) [GCC 10.3.0]
Commit hash: 22bcc7be428c94e9408f589966c2040187245d81
Installing gfpgan
Installing clip
Installing open_clip
Cloning Stable Diffusion into /kaggle/working/stable-diffusion-webui/repositories/stable-diffusion-stability-ai...
Cloning Taming Transformers into /kaggle/working/stable-diffusion-webui/repositories/taming-transformers...
Cloning K-diffusion into /kaggle/working/stable-diffusion-webui/repositories/k-diffusion...
Cloning CodeFormer into /kaggle/working/stable-diffusion-webui/repositories/CodeFormer...
Cloning BLIP into /kaggle/working/stable-diffusion-webui/repositories/BLIP...
Installing requirements for CodeFormer
Installing requirements for Web UI
Installing sd-webui-controlnet requirement: svglib

Launching Web UI with arguments: --share --xformers --deepdanbooru --lowram --theme dark --disable-safe-unpickle --disable-nan-check --ckpt /kaggle/working/st

In [9]:
# #第9步：生成图片打包复制出来
# #将生成图片打包复制出来，便于保存
# !zip -q -r output.zip /kaggle/working/stable-diffusion-webui/outputs 
# !mv /kaggle/working/stable-diffusion-webui/output.zip /kaggle/working/output.zip

# import os
# os.chdir('/kaggle/working')
# from IPython.display import FileLink
# FileLink('output.zip')

# 参考来自：https://www.kaggle.com/code/kangfilm/2-0stable-diffusion-webui-zqmod20230224/edit